In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import path

In [2]:
def dump_df(df):
    # Tem que usar essa opção pro pandas imprimir a saída toda, mas eu nn queria definir isso pro arquivo inteiro
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, "display.max_colwidth", None):
        print(df.head(100))

In [3]:
DATA_PATH = './data/glycemic_index/foods.csv'
df = pd.read_csv(DATA_PATH, delimiter=';').drop_duplicates().set_index('id')
df

,name,category,sample_amount,sample_unit,calories_kcal,calories_kj,protein,carbohydrates,fats,glycemic_index,glycemic_load
id,,,,,,,,,,,
baguette-white,Baguette (white),Baked products,100,grams,240.0,1004,7.5,49.0,2.5,95.0,46.5
baguette-whole-grain,Baguette (whole grain),Baked products,100,grams,219.0,916,6.3,38.8,1.2,73.0,28.3
bagel-wheat,Bagel (wheat),Baked products,100,grams,257.0,1075,10.0,50.5,1.6,70.0,35.4
biscuit,Biscuit,Baked products,100,grams,365.0,1527,6.2,48.5,16.5,70.0,34.0
biscotti-dry-cookies,Biscotti (dry cookies),Baked products,100,grams,358.0,1498,6.0,50.0,15.0,70.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...
tapioca-dried,Tapioca (dried),Other,100,grams,350.0,1464,0.2,87.8,0.0,85.0,74.6
taro,Taro,Other,100,grams,112.0,469,1.5,26.5,0.2,48.0,12.7
tahin,Tahin,Other,100,grams,595.0,2489,17.0,21.0,54.0,40.0,8.4


In [4]:
df.groupby('category').count()['name']

category
Baked products      49
Beverages           28
Dairy               44
Fast foods           9
Fruits              80
Grains             104
Greens              50
Meat                32
Mushrooms            2
Nuts                17
Oils and sauces     16
Other               16
Seafood              5
Soups                5
Spices              24
Sweets              42
Vegetables          55
Name: name, dtype: int64

In [5]:
# Definições base para testar modelos
from sklearn.linear_model import RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, KFold

X = df[['calories_kcal', 'carbohydrates', 'protein', 'fats']]
skf = KFold(n_splits=5)

In [6]:
y_gl = df[['glycemic_load']]

X_train, X_test, y_gl_train, y_gl_test = train_test_split(X, y_gl, test_size=0.20)

ridge = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv=skf).fit(X_train, y_gl_train)
print(f'R2 da Regressão Polinomial: {ridge.score(X_test, y_gl_test)}')

knn = KNeighborsRegressor(n_neighbors=5).fit(X_train, y_gl_train)
print(f'R2 do KNN: {knn.score(X_test, y_gl_test)}')

R2 da Regressão Polinomial: 0.6947419924424911
R2 do KNN: 0.7579633850192831


In [7]:
y_gi = df[['glycemic_index']]

X_train, X_test, y_gi_train, y_gi_test = train_test_split(X, y_gi, test_size=0.20)

ridge = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv=skf).fit(X_train, y_gi_train)
print(f'R2 da Regressão Polinomial: {ridge.score(X_test, y_gi_test)}')

knn = KNeighborsRegressor(n_neighbors=5).fit(X_train, y_gi_train)
print(f'R2 do KNN: {knn.score(X_test, y_gi_test)}')

R2 da Regressão Polinomial: 0.15501334192419614
R2 do KNN: 0.2215005108001159


In [13]:
import statsmodels.api as sm

X_c = sm.add_constant(X)
X_c_train, X_c_test, y_c_gi_train, y_c_gi_test = train_test_split(X_c, y_gi, test_size=0.20)

model2 = sm.OLS(y_c_gi_train, X_c_train)
resultado = model2.fit()
resultado.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         glycemic_index   R-squared:                       0.323
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     54.45
Date:                Wed, 14 Aug 2024   Prob (F-statistic):           1.59e-37
Time:                        22:07:48   Log-Likelihood:                -2034.1
No. Observations:                 462   AIC:                             4078.
Df Residuals:                     457   BIC:                             4099.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            35.2166      1.601     21.992      0.000      32.070      38.364
calories_kcal    -0.0228      0.025     -0.922      0.357      -0.071       0.026
carbohydrates     0.5254      0.104      5.042      0.000       0.321       0.730
protein          -0.5489      0.164     -3.346      0.001      -0.871      -0.226
fats             -0.0365      0.214     -0.171      0.865      -0.457       0.384
==============================================================================
Omnibus:                        8.059   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.018   Jarque-Bera (JB):                7.951
Skew:                           0.299   Prob(JB):                       0.0188
Kurtosis:                       3.237   Cond. No.                         487.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""